In [1]:
!python --version

Python 3.11.11


In [2]:
!pip install Cython==0.29.37  # Phiên bản mới hơn, thay vì 0.29.11
!pip install matplotlib==3.9.2  # Phiên bản mới hơn, thay vì 3.0.3
!pip install numpy==2.1.2  # Phiên bản mới hơn, thay vì 1.18.5
!pip install scapy==2.6.0  # Phiên bản mới hơn, thay vì 2.4.2
!pip install scipy==1.14.1  # Phiên bản mới hơn, thay vì 1.4.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 31.9 MB/s eta 0:00:00
  Attempting uninstall: Cython
    Found existing installation: Cython 3.0.12
    Uninstalling Cython-3.0.12:
      Successfully uninstalled Cython-3.0.12
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 70.0 MB/s eta 0:00:00
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.7.5
    Uninstalling matplotlib-3.7.5:
      Successfully uninstalled matplotlib-3.7.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 1.36.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
mlxtend 0.23.4 requires scikit-learn>=1.3.1, but you have scikit-learn 1.2.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 89.4 MB/s eta 0:00:

In [3]:
!pip uninstall -y numpy scipy scikit-learn
!pip install numpy==1.23.5 scipy==1.10.1 scikit-learn==1.2.2

Found existing installation: numpy 2.1.2
Uninstalling numpy-2.1.2:
  Successfully uninstalled numpy-2.1.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
Found existing installation: scikit-learn 1.2.2
Uninstalling scikit-learn-1.2.2:
  Successfully uninstalled scikit-learn-1.2.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 78.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 105.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mkl-umath 0.1.1 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.23.5 which is incompatible.
mkl-random 1.2.4 requires numpy<1.27.0,>=1.26.4, but you have numpy 1.

In [4]:
# Copy toàn bộ thư mục code
!cp -r /kaggle/input/codeadv/TrafficManipulator /kaggle/working/

In [5]:
import os

path = "/kaggle/working/TrafficManipulator"
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith(".pkl"):
            print(os.path.relpath(os.path.join(root, file), path))


In [6]:
import numpy as np
from sklearn.preprocessing import StandardScaler
import pickle

# Fix lỗi np.bool deprecated
import numpy
numpy.bool = bool

# Load đặc trưng benign từ file .npy
benign_features = np.load('/kaggle/working/TrafficManipulator/example/train_ben.npy')

# Khởi tạo bộ chuẩn hóa
scaler = StandardScaler()

# Huấn luyện scaler trên dữ liệu benign
scaler.fit(benign_features)

# Lưu scaler vào file .pkl
with open('/kaggle/working/TrafficManipulator/feature_normalizer.pkl', 'wb') as f:
    pickle.dump(scaler, f)

print("✅ Đã tạo xong file feature_normalizer.pkl")

✅ Đã tạo xong file feature_normalizer.pkl


In [7]:
!cp /kaggle/input/pcapdemo/benign_traffic.pcapng /kaggle/working/TrafficManipulator/malicious.pcap
!cp /kaggle/working/TrafficManipulator/example/mimic_set.npy /kaggle/working/TrafficManipulator/benign_features.npy

In [8]:
%cd /kaggle/working/TrafficManipulator/AfterImageExtractor
!python setup.py build_ext --inplace

/kaggle/working/TrafficManipulator/AfterImageExtractor
Compiling AfterImage.pyx because it changed.
[1/1] Cythonizing AfterImage.pyx
/usr/local/lib/python3.11/dist-packages/Cython/Compiler/Main.py:369: FutureWarning: Cython directive 'language_level' not set, using 2 for now (Py2). This will change in a later release! File: /kaggle/working/TrafficManipulator/AfterImageExtractor/AfterImage.pyx
  tree = Parsing.p_module(s, pxd, full_module_name)


In [9]:
%cd /kaggle/working/TrafficManipulator

/kaggle/working/TrafficManipulator


In [10]:
import sys
sys.path.insert(0, '/kaggle/working/TrafficManipulator')

In [11]:
from scapy.all import wrpcap

# Tạo danh sách gói tin rỗng
pkts = []

# Lưu file pcap rỗng
wrpcap('/kaggle/working/TrafficManipulator/_empty.pcap', pkts)
print("Đã tạo file _empty.pcap rỗng")

Đã tạo file _empty.pcap rỗng


In [12]:
file_path = '/kaggle/working/TrafficManipulator/particle.py'

with open(file_path, 'r') as f:
    content = f.read()

content = content.replace('time.clock()', 'time.perf_counter()')

with open(file_path, 'w') as f:
    f.write(content)

print("Đã thay thế time.clock() thành time.perf_counter() trong particle.py")

Đã thay thế time.clock() thành time.perf_counter() trong particle.py


In [13]:
import os

base_path = '/kaggle/working/TrafficManipulator'

for root, dirs, files in os.walk(base_path):
    for file in files:
        if file.endswith('.py'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                content = f.read()
            if 'time.clock()' in content:
                content_new = content.replace('time.clock()', 'time.perf_counter()')
                with open(file_path, 'w') as f:
                    f.write(content_new)
                print(f"Đã sửa time.clock() trong {file_path}")

Đã sửa time.clock() trong /kaggle/working/TrafficManipulator/manipulator.py


In [14]:
# from scapy.all import rdpcap, wrpcap

# # Đọc file gốc (cẩn thận với dung lượng lớn)
# packets = rdpcap('/kaggle/working/TrafficManipulator/malicious.pcap', count=10000)

# # Ghi lại file pcap mới với 50k packets
# wrpcap('/kaggle/working/TrafficManipulator/malicious_10k.pcap', packets)

# print("Đã tạo file malicious_10k.pcap với 10k gói tin")

In [15]:
# Xóa file manipulator.py cũ
!rm -f /kaggle/working/TrafficManipulator/manipulator.py

# Copy file manipulator.py mới vào
!cp /kaggle/input/ver2manipulator/manipulator.py /kaggle/working/TrafficManipulator/manipulator.py

In [16]:
# !python /kaggle/working/TrafficManipulator/main.py \
# -m /kaggle/working/TrafficManipulator/malicious_10k.pcap \
# -b /kaggle/working/TrafficManipulator/benign_features.npy \
# -n /kaggle/working/TrafficManipulator/feature_normalizer.pkl \
# -i /kaggle/working/TrafficManipulator/_empty.pcap \
# -o /kaggle/working/TrafficManipulator/statistics.pkl

In [17]:
!python /kaggle/working/TrafficManipulator/main.py \
-m /kaggle/working/TrafficManipulator/malicious.pcap \
-b /kaggle/working/TrafficManipulator/benign_features.npy \
-n /kaggle/working/TrafficManipulator/feature_normalizer.pkl \
-i /kaggle/working/TrafficManipulator/_empty.pcap \
-o /kaggle/working/TrafficManipulator/statistics.pkl

Importing AfterImage Cython Library
Importing Scapy Library
@Manipulator: Initializing ...
self.mimic_set.shape (1000, 100)
  read 262666 packets in malicious pcap
@Mani: Begin processing...
@Manipulator: Processing pkt ( 0 to 100 ) ...
--@PSO:Step 0 Finished...Global best value: 701160.7419340279
--@PSO:Step 1 Finished...Global best value: 690908.3157914294
--@PSO:Step 2 Finished...Global best value: 690577.0991874248
@Manipulator: Processing pkt ( 100 to 200 ) ...
--@PSO:Step 0 Finished...Global best value: 1085353.639249452
--@PSO:Step 1 Finished...Global best value: 1085353.639249452
--@PSO:Step 2 Finished...Global best value: 1085353.639249452
@Manipulator: Processing pkt ( 200 to 300 ) ...
--@PSO:Step 0 Finished...Global best value: 1072826.625284748
--@PSO:Step 1 Finished...Global best value: 1072826.625284748
--@PSO:Step 2 Finished...Global best value: 1072826.625284748
@Manipulator: Processing pkt ( 300 to 400 ) ...
--@PSO:Step 0 Finished...Global best value: 1039359.756978485